In [ ]:
%conda install conda-forge::tensorflow

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')

f = x*x*y + y +2

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
sess.close()

ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.


In [10]:
x1 = tf.Variable(1)
x1.graph is tf.compat.v1.get_default_graph()
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph


True

## Linear Regression with TensorFlow
- operators in TensorFlow `ops` take in a multidimensional input and outputs another one.
- tensors are multidimensional arrays.




In [ ]:
!pip install scikit-learn

In [12]:
import numpy as np
np.bool = np.bool_
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

m,n = housing.data.shape
#Adding bias term

housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")

y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
# Compute normal Equation = gradient calc
theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT,X)),XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    

In [ ]:
theta_value

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [14]:
## Batch Gradient Descent with TensorFlow
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = "y")

#generate array of random values of size n as a variable
theta = tf.Variable(tf.random.uniform([n+1,1], -1.0, 1.0), name = "theta")

#Set up all of the equations to be run as part of calculating the gradient
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
#gradients = tf.gradients(mse, [theta])[0] #-> autodiff gradients function to calculate the gradients of the MSE with respect to theta
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

# You can also use the following optimizer
#optimizer = tf.compat.v1.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

#Formula to be ran every epoch, which is the update of thetas and update the variable with .assign
#training_op = tf.compat.v1.assign(theta, theta - learning_rate * gradients)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    # Initialize the variables
    sess.run(init)
    #Every 100 epochs print the MSE
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

best_theta

Epoch 0 MSE = 8.099343
Epoch 100 MSE = 1.0134766
Epoch 200 MSE = 0.7988711
Epoch 300 MSE = 0.72122794
Epoch 400 MSE = 0.6669865
Epoch 500 MSE = 0.62782216
Epoch 600 MSE = 0.5994771
Epoch 700 MSE = 0.5789483
Epoch 800 MSE = 0.5640713
Epoch 900 MSE = 0.5532824


array([[ 2.0685525 ],
       [ 0.7952512 ],
       [ 0.16709027],
       [-0.09812938],
       [ 0.1222113 ],
       [ 0.01352751],
       [-0.04268243],
       [-0.57977915],
       [-0.54147744]], dtype=float32)

## Create a Mini-batch Gradient Descent with TensorFlow
 - replace X and y at every iteration by randomly fetching samples
 - use placeholder nodes: they dont perform actual operations, they just output the data at runtime. 


In [16]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict = {A: [[4,5,6],[7,8,9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [30]:
## Mini-Batch Gradient Descent with TensorFlow
#Save log for TensorBoard

from datetime import datetime

now = datetime.now().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    '''Fetch the mini batches one by one, then provide the value of X and y for that batch'''
    m,n = housing.data.shape
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size = batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return X_batch, y_batch

# Create X and y as placeholders since their values will be fed during the execution.
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random.uniform([n+1,1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
# Declare error and mse as a scope name `loss`
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
  c
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
#save parameters to disk
saver = tf.train.Saver()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    # Initialize the variables
    sess.run(init)
    #Every 100 epochs print the MSE
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            print(X_batch)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/tmp/SGDtf.ckpt")
        sess.run(training_op)

    best_theta = theta.eval()
    #Save final parameters to disk
    save_path = saver.save(sess, "/tmp/SGDtf_final.ckpt")
    file_writer.close()

[[ 1.00000000e+00 -1.30671795e+00 -1.08376738e+00 -6.06063433e-01
   8.43215149e-03 -3.54525297e-01  3.52006127e-02 -1.32583949e+00
   2.00136666e+00]
 [ 1.00000000e+00  9.26861525e-01  4.25936108e-01 -3.59078740e-01
  -6.87331259e-02 -3.29799809e-01 -7.92227920e-02 -9.41926503e-01
   8.18426861e-01]
 [ 1.00000000e+00 -3.54237084e-01  1.87561872e-01 -7.22593601e-01
  -3.27494254e-01 -6.37102300e-01 -1.23178456e-01  4.43905743e-01
  -1.16312209e+00]
 [ 1.00000000e+00 -5.03833692e-01 -1.30270442e-01 -3.22206136e-02
  -3.45349253e-01 -6.87436329e-01  2.41085187e-03  8.65273655e-01
  -6.59000061e-01]
 [ 1.00000000e+00  2.85206827e-01 -6.86476992e-01  2.30789867e-01
  -1.07645318e-01 -6.30037875e-01  1.73781922e-02  1.23514105e+00
  -8.43678426e-01]
 [ 1.00000000e+00 -8.59665174e-01 -7.65935071e-01 -2.77632599e-01
  -2.81741782e-01 -6.40634513e-01  5.90763819e-02  6.12452908e-01
  -2.44721566e-01]
 [ 1.00000000e+00 -8.59665174e-01 -3.68644678e-01 -4.23288318e-01
  -9.02785194e-02  5.1174982

InvalidArgumentError: Graph execution error:

Detected at node 'y_13' defined at (most recent call last):
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    File "/Users/dlebron/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    File "/Users/dlebron/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 1077, in launch_instance
    File "/Users/dlebron/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start
    File "/Users/dlebron/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/asyncio/b

In [29]:
!mkdir tmp

In [24]:
 m,n = housing.data.shape
m

20640

In [25]:
n

8